In [27]:
import os
from fastkml import kml
import zipfile

# use existing kmz ZipFile if present, otherwise open the file path
kmz_file = "L:/TO_Traffic/TMC/000 - TMC References/Google Earth Files/IntersectionControl_080421.kmz"
extraction_dir  = os.path.dirname("./maps")
with zipfile.ZipFile(kmz_file, "r") as kmz:
    kmz.extractall(extraction_dir)
import fiona
import geopandas as gpd
import pandas as pd
fp_eq = os.path.join(extraction_dir, "doc.kml")
gdf_list = []
for layer in fiona.listlayers(fp_eq) :    
    gdf = gpd.read_file(fp_eq, driver='LIBKML', layer=layer)
    gdf_list.append(gdf)

gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
gdf["long"] = gdf.geometry.x
gdf["lat"] = gdf.geometry.y
gdf.drop(columns=["Description", "geometry"], inplace=True)
gdf.to_csv("intersections.csv", index=False)

In [28]:
from glob import glob
import os
def get_files(path):
    # Get the files and not the folders absolute paths.
    # if folder, do get_files recursively
    for existing in glob(os.path.join(path, "*")):
        if os.path.isdir(existing):
            yield from get_files(existing)
        else:
            yield existing


TMC_folder_path = "L:/TO_Traffic/TMC"
all_files = []
for file in get_files(TMC_folder_path):
    all_files.append(file)


In [29]:
import json
with open("all_tmc_files.json", "w") as f:
    json.dump(all_files, f, indent=4)

# Combine all intersections

In [3]:
remaining_intersections_path = "C:\\MyApps\\TMCGis\\backend\\resources\\remaining_intersections.csv"
intersections_path = "C:\\MyApps\\TMCGis\\backend\\resources\\intersections.csv"

In [4]:
import pandas as pd
# read CSVs; fall back to latin1 if UTF-8 decoding fails
try:
    remaining_intersections = pd.read_csv(remaining_intersections_path)
except UnicodeDecodeError:
    remaining_intersections = pd.read_csv(remaining_intersections_path, encoding="latin1")

try:
    intersections = pd.read_csv(intersections_path)
except UnicodeDecodeError:
    intersections = pd.read_csv(intersections_path, encoding="latin1")

In [5]:
intersections["Signal ID"] = intersections["Name"].str.split(" ").str[-1]

In [8]:
from tqdm import tqdm

# ensure output columns exist
if "Longitude" not in remaining_intersections.columns:
    remaining_intersections["Longitude"] = pd.NA
if "Latitude" not in remaining_intersections.columns:
    remaining_intersections["Latitude"] = pd.NA

# helper to normalize signal id to comparable string
def _sid_str(x):
    if pd.isna(x):
        return None
    # convert floats like 8000100.0 -> "8000100", otherwise keep string form
    if isinstance(x, float) and x.is_integer():
        return str(int(x))
    return str(x).strip()

remaining_intersections["_sid_str"] = remaining_intersections["Signal ID"].apply(_sid_str)
intersections["_sid_str"] = intersections["Signal ID"].apply(_sid_str)

# build mapping from signal id -> (long, lat)
# build mapping from signal id -> (long, lat)
# handle non-unique _sid_str by taking the first occurrence for each id
coord_map = (
    intersections
    .dropna(subset=["_sid_str"])
    .groupby("_sid_str", sort=False)[["long", "lat"]]
    .first()
    .to_dict(orient="index")
)
# normalize keys to plain stripped strings (and filter out any None)
coord_map = {str(k).strip(): v for k, v in coord_map.items() if k is not None}

# apply mapping with a progress bar (uses index variable 'i' to avoid clobbering notebook 'idx')
for i in tqdm(remaining_intersections.index, desc="matching intersections", total=remaining_intersections.shape[0]):
    sid = remaining_intersections.at[i, "_sid_str"]
    if sid and sid in coord_map:
        remaining_intersections.at[i, "Longitude"] = coord_map[sid]["long"]
        remaining_intersections.at[i, "Latitude"] = coord_map[sid]["lat"]

# cleanup helper columns
remaining_intersections.drop(columns=["_sid_str"], inplace=True, errors="ignore")
intersections.drop(columns=["_sid_str"], inplace=True, errors="ignore")
# Drop unnamed columns if they exist
remaining_intersections = remaining_intersections.loc[:, ~remaining_intersections.columns.str.contains('^Unnamed')]
intersections = intersections.loc[:, ~intersections.columns.str.contains('^Unnamed')]


matching intersections: 100%|██████████| 8942/8942 [00:00<00:00, 50252.92it/s]


In [9]:
remaining_intersections

,Signal ID,Corridor from signal list,Intersection Name,Group,Device DNS,Vendor,IP Address,Sub Mask,Gateway,IP not by 1,Longitude,Latitude
0,8000100.0,8,CSAH 8 (West Broadway) at CSAH 10 (Bass Lake R...,NMED2,atms-8-10-switch,Cisco,192.168.29.131,255.255.255.224,192.168.29.131,NaN,-93.365631,45.054403
1,8000100.0,8,CSAH 8 (West Broadway) at CSAH 10 (Bass Lake R...,NMED2,atms-8-10-controller,Econolite,192.168.29.135,255.255.255.224,192.168.29.131,IP not by 1,-93.365631,45.054403
2,8000100.0,8,CSAH 8 (West Broadway) at CSAH 10 (Bass Lake R...,NMED2,atms-8-10-mmu,RAE,192.168.29.136,255.255.255.224,192.168.29.131,NaN,-93.365631,45.054403
3,8000100.0,8,CSAH 8 (West Broadway) at CSAH 10 (Bass Lake R...,NMED2,atms-8-10-evp,Tomar/GTT,192.168.29.137,255.255.255.224,192.168.29.131,NaN,-93.365631,45.054403
4,8000100.0,8,CSAH 8 (West Broadway) at CSAH 10 (Bass Lake R...,NMED2,atms-8-10-aps,Cambell/Polara,192.168.29.138,255.255.255.224,192.168.29.131,NaN,-93.365631,45.054403
...,...,...,...,...,...,...,...,...,...,...,...,...
8937,8000000.0,tmc,TMC Test Cabinet,Medina,atms-tmc-test-potential6,NaN,192.168.192.21,225.255.255.224,192.168.192.3,NaN,<NA>,<NA>
8938,8000000.0,tmc,TMC Test Cabinet,Medina,atms-tmc-test-potential7,NaN,192.168.192.22,225.255.255.224,192.168.192.3,NaN,<NA>,<NA>
8939,8000000.0,tmc,TMC Test Cabinet,Medina,atms-tmc-test-potential8,NaN,192.168.192.23,225.255.255.224,192.168.192.3,NaN,<NA>,<NA>
8940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>


In [10]:
import os
remaining_intersections.to_csv(os.path.dirname(remaining_intersections_path) + "\\compelete_intersections.csv", index=False)